In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import backtrader as bt

stocks = ["AMD"]

stock_data = yf.download(stocks, period = "max", interval = "1d", group_by = "tickers")
stock_data = stock_data.dropna()
signals = pd.DataFrame({"date": stock_data.index}).set_index("date")


for ticker in stocks:
    sma200 = stock_data[ticker]['Close'].rolling(window = 200).mean()
    sma50 = stock_data[ticker]['Close'].rolling(window = 50).mean()
    # stock_data[(ticker, '200_SMA')] = stock_data[ticker]['Close'].rolling(window = 200).mean()
    # stock_data[(ticker, '50_SMA')] = stock_data[ticker]['Close'].rolling(window = 50).mean()
    signal = 2* (sma50 > sma200) - 1
    signals[ticker] = signal
    stock_data[(ticker, 'signal')] = signal

stock_data.to_csv("new_data.csv")
# stock_data
signals

In [4]:
print(stock_data.columns)

MultiIndex([('AMD',   'Open'),
            ('AMD',   'High'),
            ('AMD',    'Low'),
            ('AMD',  'Close'),
            ('AMD', 'Volume'),
            ('AMD', 'signal')],
           names=['Ticker', 'Price'])


In [5]:
plt.plot(stock_data[('AMD', '200_SMA')], label = "200")
plt.plot(stock_data[('AMD', '50_SMA')], label = "50")
plt.plot(stock_data[('AMD', 'Close')])
plt.legend()
plt.show()

KeyError: ('AMD', '200_SMA')

In [3]:
class trade(bt.Strategy):
    def __init__(self):
        self.trading_logic = signals

    def next(self):
        dt = self.datas[0].datetime.date(0)
        dt = pd.to_datetime(dt)

        if dt in self.trading_logic.index:
            for i, stock in enumerate(stocks):
                signal = self.trading_logic.loc[dt, stock]

                self.order_target_percent(data = self.datas[i], target = .125 * signal)

In [27]:
class trader(bt.Strategy):
    def __init__(self):
        self.signal = self.datas[0].signal
    # def next(self):
    #     if not self.position:
    #         if self.signal[0] == 1:
    #             self.buy()  # enter long
    #     else:
    #         if self.signal[0] == -1:
    #             self.close()  # exit long

    def next(self):
        dt = self.datas[0].datetime.date(0)
        dt = pd.to_datetime(dt)

        if dt in self.trading_logic.index:
            for i, stock in enumerate(stocks):
                cash = self.broker.getcash()
                price = self.data.close[0]
                size = int(cash//price)

                if self.signal[0] == 1:  # Buy signal
                    if self.position.size < 0:  # If short, close it
                        self.close()
                    self.buy(size = size)  # Go long
                elif self.signal[0] == -1:  # Sell signal
                    if self.position.size > 0:  # If long, close it
                        self.close()
                    self.sell(size = size)  # Go short


In [29]:
cerebro = bt.Cerebro()

# Load your DataFrame into Backtrader
data = bt.feeds.PandasData(dataname=stock_data)
cerebro.adddata(data)

# Add the strategy
cerebro.addstrategy(trader)

# Set initial capital
cerebro.broker.setcash(100000.0)

# Add analyzers if desired (optional)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Run the backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(style = 'candlestick')


Starting Portfolio Value: 100000.00


AttributeError: 'tuple' object has no attribute 'lower'

In [ ]:
cerebro = bt.Cerebro()

for stock in stocks:
    feed = bt.feeds.PandasData(dataname = stock_data[stock])
    cerebro.adddata(data = feed)

cerebro.broker.set_cash(10_000)
cerebro.addstrategy(trade)

results = cerebro.run()[0]
strategy = results[0]

final_value = cerebro.broker.get_value()
print(final_value)


cerebro.plot()[0][0].savefig("test.png")
cerebro.plot(style = 'candlestick')[0][0].savefig("new.png")

AttributeError: 'Lines_LineSeries_DataSeries_OHLC_OHLCDateTime_Abst' object has no attribute 'signal'